In [1]:
import ee
import json
import geojson
import pandas_gbq as gbq
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
from shapely import wkt

params to Zonal Stats Function:

- service account required
- service account key file required
- sql query required
- bigquery project_id required
- Image Collection URL required
- start_date required
- end_date required
- stat required (mean, std)*args
- zonal config (
    - scale
    - crs
    - crsTransform
    - bestEffort
    - maxPixels
    - tileScale
) **kwargs
- band_calculation=**kwargs non-required


1. Pull from Big Query ()
2. What is the geometry field type? (ie shapely, json, geojson, wkt)
    - Standardize
3. Create bounding box if total area is small; otherwise use individual polygons as aois
4. Create ee.Geometry.Polygon


In [ ]:
def bigqee_zonalstats(
        service_account,
        service_account_key_file,
        BigQuery_projectID,
        BigQuery_SQL,
        image_collection,
        start_date,
        end_date,
        stats=['mean', 'median', 'max', 'min', 'stdDev', 'sum'],
        band_calc={},
):
        """
        This function uses Google Earth Engine to calculate zonal statistics for a
        given BigQuery SQL query. The function returns a GeoJSON object with the
        zonal statistics.
        
        Parameters
        ----------
        service_account : str
                The service account name to be used for authentication.
        service_account_key_file : str
                The path to the service account key file.
        BigQuery_projectID : str
                The BigQuery project ID.
        BigQuery_SQL : str
                The BigQuery SQL query.
        image_collection : str
                The Earth Engine image collection URL to be used for zonal statistics.
        start_date : str
                The start date for the Earth Engine image collection (inclusive).
        end_date : str
                The end date for the Earth Engine image collection (Exclusive).

        stats : list
                The list of zonal statistics to be calculated. The options are:
                ['mean', 'median', 'max', 'min', 'stdDev', 'sum'].

        band_calc : dict
                The dictionary possible band calcualtions. The default is
                NDVI: 'NDVI = (NIR - RED) / (NIR + RED)'. {'NDVI': [NIR_Band, RED_Band]}

        Returns
        -------
        geojson : object
                The GeoJSON object with the zonal statistics.
        """
        import ee
        import geojson
        import pandas_gbq as gbq
        import geopandas as gpd
        import pandas as pd
        from shapely.geometry import shape
        from shapely import wkt
        
        # Authenticate to Google Cloud
        credentials = ee.ServiceAccountCredentials(
                service_account,
                service_account_key_file
        )
        ee.Initialize(credentials)
        
        # Get the BigQuery table
        table = gbq.read_gbq(
                BigQuery_SQL,
                project_id=BigQuery_projectID,
                dialect='standard'
        )
        
        # if table is empty, throw error
        if table.empty:
                raise ValueError('The BigQuery table is empty.')
        
        # detect the geometry column
        geometry_column = None
        for column in table.columns:
                if column.lower() == 'geometry':
                        geometry_column = column
                        break
        if geometry_column is None:
                raise ValueError('The BigQuery table does not have a geometry column.')
        
        # convert the geometry column to GeoJSON
        table['geometry'] = table[geometry_column].apply(lambda x: geojson.Feature(geometry=wkt.loads(x)))

        # loop through the stats and calculate zonal stats on each feature in table
        # initialize the image collection
        image_collection = ee.ImageCollection(image_collection)\
                .filterDate(start_date, end_date)

        # check if band_calc is empty
        if band_calc:
        # conduct NDVI calculation
                bands = [val for val in band_calc.values()][0]
                image_collection = image_collection.normalizedDifference(bands)

        for stat in stats:
                options = defaultdict(lambda: ee.Reducer.sum(),
                                {
                                        'mean': ee.Reducer.mean(), 
                                        'median': ee.Reducer.median(),
                                        'max': ee.Reducer.max(),
                                        'min': ee.Reducer.min(),
                                        'stdDev': ee.Reducer.stdDev()
                                }
                )
                # get the reducer
                reducer = options[stat]
                        
                # loop through the table and calculate zonal stats
                for index, row in table.iterrows():
                        # convert the GeoJSON feature to an Earth Engine geometry
                        zone = ee.Geometry.Polygon(row['geometry']['coordinates'])
                        aoi_col = image_collection.filterBounds(zone)


                        # check if aoi_cal is empty
                        if not aoi_col.getInfo():
                                raise ValueError(f'The image collection is empty for bounding feature {index}.')
                        
                        # calculate zonal stats
                        zonal_stats = aoi_col.reduceRegion(
                                reducer=reducer,
                                geometry=zone,
                                scale=scale,
                                maxPixels=1e13
                        ).getInfo()
                        # create df from stats_dict
                        df = pd.DataFrame(zonal_stats, index=[0])
                        band_df = pd.concat([band_df, df], axis=0, ignore_index=True)
        
        
        

In [27]:
from collections import defaultdict

In [31]:

something = 'mean'
options = defaultdict(lambda: 4, {'mean': ee.Reducer.mean(), 
                                  'median': ee.Reducer.median(),
                                    'max': ee.Reducer.max(),
                                    'min': ee.Reducer.min(),
                                    'stdDev': ee.Reducer.stdDev(),
                                    'sum': ee.Reducer.sum()}
                                    )

for i in range(1, 1000000):
    the_thing = options[something]

print(the_thing)


ee.Reducer({
  "functionInvocationValue": {
    "functionName": "Reducer.mean",
    "arguments": {}
  }
})


In [2]:
service_account = 'austinbreunig@ee-obia-waterbodies.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'ee-obia-waterbodies-33e289809d3b.json')
ee.Initialize(credentials)

In [3]:
sql = """
SELECT * FROM `ee-obia-waterbodies.ee_bq_test.ss_total_cat_depths`
"""

df = gbq.read_gbq(sql, project_id='ee-obia-waterbodies', dialect='standard')

Downloading: 100%|██████████|


In [4]:
df.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_
0,"{""type"":""Polygon"",""coordinates"":[[[-95.4399074...",17056189,4,Very High,48039,0,5,10,15,18,33
1,"{""type"":""Polygon"",""coordinates"":[[[-95.4374074...",17056827,4,Very High,48039,0,5,10,15,18,33
2,"{""type"":""Polygon"",""coordinates"":[[[-95.4374074...",17057446,4,Very High,48039,0,5,10,15,19,34
3,"{""type"":""Polygon"",""coordinates"":[[[-95.4407407...",17057591,4,Very High,48039,0,5,10,15,19,34
4,"{""type"":""Polygon"",""coordinates"":[[[-95.4396296...",17057779,4,Very High,48039,0,5,10,15,19,34


In [ ]:
df['geom']

In [5]:
def json_to_shapely(geometry):
    """
    Convert JSON geometry to WKT format.
    
    Args:
        json_geometry (dict): JSON geometry object.
    
    Returns:
        str: WKT representation of the geometry.
    """
    shapely_geometry = shape(json.loads(geometry))
    return shapely_geometry.wkt


In [6]:
gdf = df.copy()

In [7]:

gdf['geom'] = gdf['geom'].apply(json_to_shapely)

In [13]:
geojson.Feature(geometry=wkt.loads(gdf.iloc[0]['geom']))

{"geometry": {"coordinates": [[[-95.439907, 28.966759], [-95.440093, 28.966759], [-95.440093, 28.966667], [-95.440648, 28.966667], [-95.440648, 28.966574], [-95.439907, 28.966574], [-95.43963, 28.966574], [-95.43963, 28.966481], [-95.43963, 28.966389], [-95.439907, 28.966389], [-95.439907, 28.966481], [-95.440093, 28.966481], [-95.440093, 28.966389], [-95.44037, 28.966389], [-95.44037, 28.966296], [-95.43963, 28.966296], [-95.439352, 28.966296], [-95.439352, 28.966204], [-95.439444, 28.966204], [-95.439444, 28.966111], [-95.439815, 28.966111], [-95.439815, 28.966019], [-95.439352, 28.966019], [-95.439352, 28.965833], [-95.43963, 28.965833], [-95.43963, 28.965926], [-95.439722, 28.965926], [-95.439722, 28.965741], [-95.439907, 28.965741], [-95.439907, 28.965926], [-95.44, 28.965926], [-95.44, 28.965833], [-95.440185, 28.965833], [-95.440185, 28.965926], [-95.44037, 28.965926], [-95.44037, 28.965741], [-95.440278, 28.965741], [-95.440278, 28.964352], [-95.440463, 28.964352], [-95.440463,

In [44]:
# shapely to wkt geom to geojson
gdf['geom'] = gdf['geom'].apply(lambda x: geojson.Feature(geometry=x))

In [45]:
gdf.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_
0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056189,4,Very High,48039,0,5,10,15,18,33
1,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056827,4,Very High,48039,0,5,10,15,18,33
2,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057446,4,Very High,48039,0,5,10,15,19,34
3,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057591,4,Very High,48039,0,5,10,15,19,34
4,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057779,4,Very High,48039,0,5,10,15,19,34


In [46]:
all_features = []
for index, row in gdf.iterrows():
    all_features.append(row['geom']['geometry']['coordinates'])

aoi = ee.Geometry.MultiPolygon(all_features)

In [47]:
col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [48]:
aoi_col = col.filterBounds(aoi)
image = aoi_col.filterDate('2023-06-01', '2023-06-30').first()

# ndvi calculation
ndvi = image.normalizedDifference(['B8', 'B4'])

# check if a dict is empty
if not ndvi.getInfo():
    print('empty')


In [49]:
band_df = pd.DataFrame()
for index, row in gdf.iterrows():
    coords = row['geom']['geometry']['coordinates']
    zone = ee.FeatureCollection(ee.Geometry.Polygon(coords))

    stats_dict = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=zone,
    scale=30
).getInfo()
    
    # create df from stats_dict
    df = pd.DataFrame(stats_dict, index=[0])
    band_df = pd.concat([band_df, df], axis=0, ignore_index=True)



    

In [50]:
band_df.head()

,nd
0,0.845340
1,0.826523
2,0.813726
3,0.816328
4,0.840232


In [51]:
gdf = gdf.join(band_df)


In [52]:
gdf.head()

,geom,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_,nd
0,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056189,4,Very High,48039,0,5,10,15,18,33,0.845340
1,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17056827,4,Very High,48039,0,5,10,15,18,33,0.826523
2,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057446,4,Very High,48039,0,5,10,15,19,34,0.813726
3,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057591,4,Very High,48039,0,5,10,15,19,34,0.816328
4,"{'type': 'Feature', 'geometry': {'type': 'Poly...",17057779,4,Very High,48039,0,5,10,15,19,34,0.840232


In [207]:
gdf.columns

Index(['geom', 'ID', 'RISK_VALUE', 'SURGE_RISK', 'FIPSSTCO', 'CAT1DEP',
       'CAT2DEP', 'CAT3DEP', 'CAT4DEP', 'CAT5DEP', 'total_cat_', 'nd'],
      dtype='object')

In [208]:
gdf['geom'] = gdf['geom'].apply(lambda x: shape(x['geometry']))

In [183]:
gpd.GeoDataFrame(gdf, geometry='geom', crs=4326).to_file('ndvi.shp')

In [152]:
image

In [191]:
ndvi.getInfo()

{'type': 'Image',
 'bands': [{'id': 'nd',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -1,
    'max': 1},
   'dimensions': [7741, 7881],
   'crs': 'EPSG:32615',
   'crs_transform': [30, 0, 174585, 0, -30, 3313215]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-95.882442745473, 29.916241781187402],
    [-95.88466594089704, 29.91619304537997],
    [-95.99455146212536, 29.476648225776238],
    [-96.10100454522356, 29.04777132195221],
    [-96.20674263451791, 28.61885317312932],
    [-96.31199552970635, 28.18906585924985],
    [-96.31065331966452, 28.188511353160738],
    [-95.53378332512604, 28.03584494684638],
    [-94.42392482234081, 27.809210544984897],
    [-94.39827630112785, 27.90679780616],
    [-94.3555008139391, 28.06928519217053],
    [-94.32005718871515, 28.203663466025045],
    [-94.27882232785078, 28.359612623380922],
    [-94.24452219386873, 28.48908590532632],
    [-94.17790059614084, 28.739864837933563],
   